In [1]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

%cd '/content/gdrive/My Drive/LDS0_K273_ONLINE_DoThiPhuong/Topic_3/'

Mounted at /content/gdrive
/content/gdrive/My Drive/LDS0_K273_ONLINE_DoThiPhuong/Topic_3


In [2]:
! pip install underthesea

     |████████████████████████████████| 7.6 MB 16.2 MB/s 
     |████████████████████████████████| 743 kB 54.5 MB/s 
     |████████████████████████████████| 581 kB 55.6 MB/s 
     |████████████████████████████████| 235 kB 59.7 MB/s 


In [3]:
! pip install demoji

     |████████████████████████████████| 42 kB 1.5 MB/s 


In [4]:
! pip install pyvi

     |████████████████████████████████| 8.5 MB 16.1 MB/s 


In [5]:
import pandas as pd
import numpy as np
from underthesea import word_tokenize, pos_tag, sent_tokenize
import regex
import demoji
from pyvi import ViPosTagger, ViTokenizer
import string
import Topic3_Lib as func

In [6]:
from datetime import datetime

In [7]:
example = '''Đầu năm vk ck tôi đi ctác ở sg ít ngày. 
Chúng tôi ở ks XXX. Ks ổn 👍, 😊. 
Phục vụ tốt chăm sóc nhiệt tình. 
Buffet ngon nhiều món. Phòng sạch sẽ.đầy đủ tiện nghi, view đẹp hướng ra sông sg.  
Nhân viên thái độ phục vụ tốt 💙. Vị trí trung tâm tiện cho di chuyển thăm quan.'''

In [8]:
document = func.clean_text_thesea(example, func.emoji_dict, func.teen_dict, func.wrong_lst, func.stopwords_lst)

In [9]:
document

'công_tác ổn thích cười tốt đầy_đủ tiện đẹp phục_vụ tốt yêu tiện di_chuyển thăm'

In [10]:
# read data
data = pd.read_csv('foody_08032022.csv')

In [11]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22734 entries, 0 to 22733
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   restaurant    22734 non-null  object 
 1   review_text   22734 non-null  object 
 2   review_score  22734 non-null  float64
dtypes: float64(1), object(2)
memory usage: 533.0+ KB


In [12]:
data = data.drop_duplicates()
data.shape

(22734, 3)

In [13]:
data[data.isna().T.any()]

,restaurant,review_text,review_score


In [14]:
data[data.isnull().T.any()]

,restaurant,review_text,review_score


Không có dữ liệu trùng/NaN/null

In [15]:
data['review_text_1'] = data.review_text.apply(lambda x: func.clean_text_thesea(str(x), func.emoji_dict, func.teen_dict, func.wrong_lst, func.stopwords_lst))

In [16]:
data.head()

,restaurant,review_text,review_score,review_text_1
0,Chả Cá Hà Nội Xưa,"Nhà hàng mới đổi địa chỉ sang 24 Hồng Hà, khá ...",8.8,đổi địa_chỉ khá uống cực khá ổn chuẩn laịii
1,Chả Cá Hà Nội Xưa,Quán đã chuyển về 22 Hồng Hà cới phong cách kh...,3.2,cới đổi tệ nhất ngâm tím không mùi_vị đắt nâng...
2,Chả Cá Hà Nội Xưa,"Giá niêm yết trên foody một kiểu, giá lúc ship...",3.2,niêm_yết shipper kêu bực_mình không tiếc đặt_h...
3,Chả Cá Hà Nội Xưa,Xem review thấy mọi người khen chả cá lã vọng ...,7.0,review chả_cá vọng vọng ăn_giá_cả đợi không kh...
4,Chả Cá Hà Nội Xưa,Tối nay mới đi ăn quán này món chả cá lã vọng ...,9.2,vọng đậu khá không đắt khá ân_cần vọng kêu thử...


In [17]:
data.tail()

,restaurant,review_text,review_score,review_text_1
22729,Rodstarz FastFood Restaurant - Trương Công Định,Hôm nay trời mưa hội bạn của mình rủ đi ăn món...,7.0,mưa rủ béo béo nóng nóng phô liền lôi ngậm phô...
22730,Rodstarz FastFood Restaurant - Trương Công Định,Lâu lâu chạy ngang thấy quán có món ăn mới nên...,7.0,chạy tấp liền thử béo nghiện test thử thiệc tư...
22731,Rodstarz FastFood Restaurant - Trương Công Định,"Vị trí thuận lợi dễ tìm, tuy vậy không gian kh...",7.8,thuận_lợi không khá hẹp tặng vị khẩu_vị mặn ch...
22732,Rodstarz FastFood Restaurant - Trương Công Định,"Vào ""ăn sáng"" nhưng được giá cơm trưa. Chị nhâ...",8.4,giòn ổn rán kèm không phô khônh lắm ổn giảm_gi...
22733,Rodstarz FastFood Restaurant - Trương Công Định,"Địa điểm quen thuộc của nhóm mình, bọn mình ăn...",7.0,giá_cả rẻ sạch_đẹp ưa_thích


In [18]:
data = data[data.review_text_1 != ' ']
data = data[data.review_text_1 != '']
data.shape

(22296, 4)

Gắn nhãn cho dữ liệu với:
- review_score < 6.5 là dislike (label 0)
- review_score >= 6.5 là like (label 1)

In [19]:
data['label'] = data.review_score.apply(lambda x: 0 if x < 6.5 else 1)
data.iloc[10:20, :]

,restaurant,review_text,review_score,review_text_1,label
10,Phúc Long Đinh Tiên Hoàng,Đặt đơn bôm đợt 12/12 nên được tặng cái hold c...,8.8,bôm tặng khá mê đậm uống uống khó lớn uống xỉu...,1
11,Phúc Long Đinh Tiên Hoàng,Mùa dịch k cho khách ngồi lại mới thấy quán rộ...,7.6,dịch không đẹp_đẽ thủ tươi đậm khó uống không ...,1
12,Phúc Long Đinh Tiên Hoàng,"Chi nhánh pha ok. Mình gọi trà ô long dâu, bỏ ...",7.2,uống,1
13,Phúc Long Đinh Tiên Hoàng,Thứ 1: đặt trà nhãn thì làm trà nhãn bỏ toppin...,4.0,thứ_nguyên không rửa vệ chế,0
14,Phúc Long Đinh Tiên Hoàng,2 lần gần nhất mình đến đây đều có trải nghiệm...,2.6,nhất gửi gửi tốn_phí đứng đợi không nổi nóng t...,0
15,Phúc Long Đinh Tiên Hoàng,Đầu tháng mình đặt ly trà thảo mộc và trà nhãn...,4.2,thảo_mộc nhãn trà_chát uống không nổi đãi uống...,0
16,Phúc Long Đinh Tiên Hoàng,Hôm nay tôi cùng với bạn bè họp nhóm ở đây. Lầ...,2.6,họp gửi thái_độ tệ gửi kiên_quyết không lừa gử...,0
17,Phúc Long Đinh Tiên Hoàng,Chi nhánh này mình hay ghé lắm. Gần nhà mình c...,8.0,ghé lắm trầnn khải thích ghé không rộng đông đ...,1
18,Phúc Long Đinh Tiên Hoàng,Trải nghiệm 1 lần cafe tại Phúc Long là một lầ...,1.4,không_thể_nào quên quyết_tâm phúc chạy phúc bả...,0
19,Phúc Long Đinh Tiên Hoàng,Nhạc bật quá to mặc dù đã yêu cầu giảm âm lượn...,1.0,bật mặc_dù giảm ráng bật,0


In [20]:
data = data[['restaurant', 'review_text', 'review_score', 'review_text_1', 'label']]
data.to_csv('foody_08032022pre.csv', index = False)

In [21]:
data = pd.read_csv('foody_08032022pre.csv')

In [22]:
data.head()

,restaurant,review_text,review_score,review_text_1,label
0,Chả Cá Hà Nội Xưa,"Nhà hàng mới đổi địa chỉ sang 24 Hồng Hà, khá ...",8.8,đổi địa_chỉ khá uống cực khá ổn chuẩn laịii,1
1,Chả Cá Hà Nội Xưa,Quán đã chuyển về 22 Hồng Hà cới phong cách kh...,3.2,cới đổi tệ nhất ngâm tím không mùi_vị đắt nâng...,0
2,Chả Cá Hà Nội Xưa,"Giá niêm yết trên foody một kiểu, giá lúc ship...",3.2,niêm_yết shipper kêu bực_mình không tiếc đặt_h...,0
3,Chả Cá Hà Nội Xưa,Xem review thấy mọi người khen chả cá lã vọng ...,7.0,review chả_cá vọng vọng ăn_giá_cả đợi không kh...,1
4,Chả Cá Hà Nội Xưa,Tối nay mới đi ăn quán này món chả cá lã vọng ...,9.2,vọng đậu khá không đắt khá ân_cần vọng kêu thử...,1


In [23]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22296 entries, 0 to 22295
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   restaurant     22296 non-null  object 
 1   review_text    22296 non-null  object 
 2   review_score   22296 non-null  float64
 3   review_text_1  22296 non-null  object 
 4   label          22296 non-null  int64  
dtypes: float64(1), int64(1), object(3)
memory usage: 871.1+ KB


In [24]:
data.tail()

,restaurant,review_text,review_score,review_text_1,label
22291,Rodstarz FastFood Restaurant - Trương Công Định,Hôm nay trời mưa hội bạn của mình rủ đi ăn món...,7.0,mưa rủ béo béo nóng nóng phô liền lôi ngậm phô...,1
22292,Rodstarz FastFood Restaurant - Trương Công Định,Lâu lâu chạy ngang thấy quán có món ăn mới nên...,7.0,chạy tấp liền thử béo nghiện test thử thiệc tư...,1
22293,Rodstarz FastFood Restaurant - Trương Công Định,"Vị trí thuận lợi dễ tìm, tuy vậy không gian kh...",7.8,thuận_lợi không khá hẹp tặng vị khẩu_vị mặn ch...,1
22294,Rodstarz FastFood Restaurant - Trương Công Định,"Vào ""ăn sáng"" nhưng được giá cơm trưa. Chị nhâ...",8.4,giòn ổn rán kèm không phô khônh lắm ổn giảm_gi...,1
22295,Rodstarz FastFood Restaurant - Trương Công Định,"Địa điểm quen thuộc của nhóm mình, bọn mình ăn...",7.0,giá_cả rẻ sạch_đẹp ưa_thích,1
